In [ ]:
# @title set environment
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2

In [ ]:
# @title PLA algorithm
def PLA_no_bias(X, y, number_of_generations ):
    N,d = X.shape
    normal = np.random.randn(d,)
    
    for _ in range(number_of_generations):
        # find one wrong sample
        for i in range(N):
            if np.dot(X[i], normal) * y[i] <= 0:
                normal += y[i] * X[i]
                # print accuracy
                if _ % 100 == 99:
                    print("Iter %s: %s"
                            %(_+1, (np.sign(np.dot(X, normal)) == y).mean()))
                break
        else:
            break

    #print((np.sign(np.dot(X, normal)) == y).mean())
    return normal

def PLA_with_bias(X, y, number_of_generations):
    N,d = X.shape
    augX = np.hstack([np.ones((N,1)), X])
    bias_normal = PLA_no_bias(augX, y, number_of_generations)
    return bias_normal[0], bias_normal[1:]

In [ ]:
# @title read dataset
path = "./Dataset/File_list.txt"
image_data=[]
lable_data=[]
with open(path) as f:
    text=f.read()
    list=text.split('\n')
for line in list:
    img = cv2.imread(line, cv2.IMREAD_GRAYSCALE)
    img=np.reshape(img, (1, 900))
    
    if((line[10]=='M')):
        image_data.append(img[0])
        lable_data.append(1)
    if((line[10]=='H')):
        image_data.append(img[0])
        lable_data.append(-1)
image_data=np.array(image_data)
lable_data=np.array(lable_data)

In [ ]:
# @title Generration aimation
fig,imgplot= plt.subplots() 
def init():
    bias,normal=PLA_with_bias(image_data, lable_data, 0)
    weight=normal
    weight=np.reshape(weight, (30, 30))
    imgplot = plt.imshow(weight,cmap='gray')
def run(data):
    bias,normal=PLA_with_bias(image_data, lable_data, data)
    weight=normal
    weight=np.reshape(weight, (30, 30))
    imgplot = plt.imshow(weight,cmap='gray')
ani = animation.FuncAnimation(fig, run, frames=100, init_func=init) 
ani.save('generation_animation.gif', fps=5)   # 儲存為 gif
plt.show()

In [ ]:
# @title ditect setting
number_of_generation=1000 # @param {type:"number"}
bias,normal=PLA_with_bias(image_data, lable_data, number_of_generation)

In [ ]:
# @title ditect the image
User_path="Test.png" # @param {type:"string"}
img = cv2.imread(User_path, cv2.IMREAD_GRAYSCALE)
imgplot = plt.imshow(img,cmap='gray')
plt.show(imgplot)
img = np.reshape(img,(1, 900))[0]
if(np.dot(img,normal)-bias>0):
    print("This is a picture of monkey")
else:
    print("This is a picture of human")